## **Modules**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime, os
import random

from pandas import DataFrame

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LeakyReLU, Dense, Embedding, Dropout, LSTM, Input, BatchNormalization, Flatten, Bidirectional, Reshape, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
TF_ENABLE_ONEDNN_OPTS=0

2022-06-16 22:09:39.074206: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-16 22:09:39.174538: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2022-06-16 22:09:39.687974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-06-16 22:09:39.688028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-06-16 22:09:39.688033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


2022-06-16 22:09:40.799114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:988] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:09:40.842951: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:988] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:09:40.843394: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:988] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
%load_ext tensorboard

In [3]:
%tensorboard --logdir=logs

## **Data**

In [4]:
# file_path = "../input/data-with-features-ver01/data_with_features_ver01.csv"
file_path = "data_with_features_ver01.csv"
df = pd.read_csv(file_path)
df

,Unnamed: 0,Time_UTC_Start,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,smoothed_minus_DM72,mad72,ATR72,CCI72,plus_DI72,minus_DI72,DX72,ADX72,BOP72,MFI72
0,0,2018-07-24 09:00:00,7991.55,7995.89,7991.52,7994.86,31.547768,2.521824e+05,114.0,17.411796,...,728.673056,161.526701,8.191528,174.835387,16870.670917,8895.447532,30.952366,11.256566,0.141162,55.823217
1,1,2018-07-24 10:00:00,8022.05,8032.85,8022.05,8032.85,11.358339,9.117570e+04,136.0,7.730805,...,689.193056,165.613529,8.218889,180.754117,17529.241584,8385.477220,35.284058,11.590281,0.147690,56.511624
2,2,2018-07-24 11:00:00,8140.18,8145.00,8135.00,8135.00,61.441457,5.001102e+05,296.0,29.340389,...,667.261944,170.471458,8.304167,213.721125,19586.447566,8035.266767,41.819203,12.010127,0.133587,56.879834
3,3,2018-07-24 12:00:00,8257.98,8266.61,8255.02,8263.01,36.020530,2.976317e+05,317.0,16.279052,...,667.261944,176.651978,8.305417,247.779179,20948.826903,8034.057426,44.559987,12.462209,0.138612,57.654664
4,4,2018-07-24 13:00:00,8218.71,8228.68,8216.45,8220.00,59.458543,4.887918e+05,284.0,31.777879,...,743.866250,181.885078,8.400694,221.563167,19001.025048,8854.818550,36.423979,12.795011,0.126266,56.079662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33186,33186,2022-05-07 03:00:00,35953.63,35953.64,35943.60,35951.24,6.376430,2.292318e+05,339.0,3.818980,...,7035.463472,1482.198889,45.395556,-79.503234,9943.275773,15498.132771,21.833921,13.697214,0.042153,61.134462
33187,33187,2022-05-07 04:00:00,35812.22,35826.14,35812.21,35826.14,6.066650,2.173069e+05,348.0,5.129450,...,7222.968611,1503.375019,45.295139,-82.701137,9828.032810,15946.454274,23.738286,13.836674,0.044086,60.964637
33188,33188,2022-05-07 05:00:00,35746.62,35768.06,35722.24,35768.05,34.156320,1.220607e+06,993.0,19.901990,...,7251.375139,1524.164087,45.760833,-83.214330,9728.015831,15846.248004,23.923395,13.976767,0.055374,61.167920
33189,33189,2022-05-07 06:00:00,35898.96,35903.80,35890.03,35900.00,16.707700,5.997658e+05,580.0,10.902570,...,7161.405139,1539.731304,45.436111,-74.759128,10145.099652,15761.483463,21.679369,14.083748,0.045734,61.377534


#### Test naive method:
In this method, we use volume_SMA4 and volume_SMA8 for buying signal. If current volume_SMA4 > previous volume_SMA8 and the previous volume_SMA4 < the previous volume of the previous volume_SMA8, we buy BTC. The accuracy result
is the possibility that the price will go up in the next row after buying.

In [5]:
a = df[(df["volume_SMA4"] >= df["volume_SMA8"])].index
b = df[(df["volume_SMA4"] < df["volume_SMA8"])].index+1
a_b = set(a).intersection(set(b))
lst = sorted(list(a_b))
print((df["close"].to_numpy()[np.array(lst)+1] - df["close"].to_numpy()[lst] >= 0).sum()/len(lst))

0.5124072652852392


### **Preprocessing**
Keep original_df as the original dataframe while df holds the dataframe to be changed in the next data processing phases.

In [6]:
original_df = pd.read_csv(file_path)
original_df["price_diff"] = original_df["close"].copy()
original_df.drop(columns=["Unnamed: 0", "Time_UTC_Start", "close"], inplace=True)
original_df.dropna()
df = original_df.copy()

Create features for timestamp

In [7]:
# df["day"] = pd.to_datetime(df["Time_UTC_Start"]).dt.day/31
# df["month"] = pd.to_datetime(df["Time_UTC_Start"]).dt.month/12
# df["year"] = pd.to_datetime(df["Time_UTC_Start"]).dt.year/2018
# df["hour"] = pd.to_datetime(df["Time_UTC_Start"]).dt.hour/24

# if diff_percentage == True:
#     df["day"] /= 100
#     df["month"] /= 100
#     df["year"] /= 100
#     df["hour"] /= 100

### Data normalization

In [8]:
def data_normalization(df: DataFrame, days_range: int=20, include_cur_row: bool=False):
    '''
    Return the normalized data:
    + df: the dataframe to be normalized
    + range: the number of previous rows (or including the current row) to be considered in the normalization
    + include_cur_row: True if we consider the current row in the normalization process (calculate mean and std
    using the current row and (range-1) previous rows), False if we want to use all the passed data for normalization 
    processing ((calculate mean and std using (range) previous rows))
    '''
    
    df_roll = None

    if include_cur_row == False:
        df_roll = df.rolling(days_range, closed='left')
    else:
        df_roll = df.rolling(days_range)
        
    res_df = (df - df_roll.mean()) / df_roll.std()
    res_df.replace([np.inf, -np.inf], 0, inplace=True)
    res_df.dropna(inplace=True)
    res_df.reset_index(drop=True, inplace=True)
    return res_df


def data_denormalization(predictions, original_df: DataFrame, steps: int, col_idx: int, days_range: int=20, include_cur_row: bool=False):
    '''
    Return the rescaled predictions data:
    + df: the dataframe to be denormalized
    + original_df: the dataframe used for denormalizing df before, the dataframe length must be greater than the
    predictions length with a value of 'days_range' because we need 'days_range' rows before to calculate the
    standard deviation and mean to be used for denormalization process.
    + range: the number of previous rows (or including the current row) to be considered in the denormalization
    + include_cur_row: True if we consider the current row in the denormalization process (calculate mean and std
    using the current row and (range-1) previous rows), False if we want to use all the passed data for denormalization 
    processing ((calculate mean and std using (range) previous rows))
    '''
    
    df_roll = None
    if include_cur_row == False:
        df_roll = original_df.rolling(days_range, closed='left')
    else:
        df_roll = original_df.rolling(days_range)
    
    res_df = original_df.copy()
    res_df.iloc[days_range+steps:,col_idx] = predictions.reshape((-1))
    res_df = res_df * df_roll.std() + df_roll.mean()

    return res_df.iloc[days_range+steps:,col_idx]

In [9]:
df = data_normalization(df)

The length of df is less than the length of original_df by 'days_range' value since we need 'days_range' first rows for calculating standard deviation and mean

In [10]:
norm_train_df = df.iloc[0:int(len(df)*0.7),:]

# Include 20 additional rows before for calculating std and mean
train_df = original_df.iloc[:int(len(df)*0.7)+20,:]
train_data = norm_train_df.to_numpy()
num_rows_train, num_cols_train = train_data.shape

####################################################################

norm_val_df = df.iloc[int(len(df)*0.7):int(len(df)*0.8),:]

# Include 20 additional rows before for calculating std and mean
val_df = original_df.iloc[int(len(df)*0.7):int(len(df)*0.8)+20,:]

val_data = norm_val_df.to_numpy()
num_rows_val, num_cols_val = val_data.shape

####################################################################

norm_test_df = df.iloc[int(len(df)*0.8):]

# Include 20 additional rows before for calculating std and mean
test_df = original_df.iloc[int(len(df)*0.8):,:]
test_data = norm_test_df.to_numpy()
num_rows_test, num_cols_test = test_data.shape

In [11]:
# Cast type
train_data = np.asarray(norm_train_df).astype('float32')
val_data = np.asarray(norm_val_df).astype('float32')
test_data = np.asarray(norm_test_df).astype('float32')

In [12]:
# Define the steps or sequence length for used in regression training process. We need to have 'steps' rows from
# the current price to predict the next price
steps = 8

### Split sequence by window size
Split sequences to train set and test set

In [13]:
def split_sequences(data, window_size, price_diff_col_idx):
    x = []
    y = []
    for i in range(0,len(data)-window_size):
        x.append(data[i:i+window_size,:]) # Take window_size rows data before
        y.append(data[i+window_size,price_diff_col_idx]) # To predict the current value of label colume
    return np.array(x), np.array(y).reshape((-1,1))

### Split sequence for GAN training
Split sequences to train GAN model, we have 2 kinds of time series for discriminator:
+ The true time series is the time series taken in split_sequences
+ The false time series is the time series taking a time series of 'steps' length and include the next predicted price
The discriminator will identify these 2 kinds of time series

In [14]:
def split_sequences_gan(data, window_size, price_diff_col_idx):
    x = []
    for i in range(0,len(data)-window_size):
        x.append(data[i:i+window_size+1,price_diff_col_idx])
    return np.array(x)

+ x_train_gan is used for include real time series for discriminator
+ x_train, y_train is used for training phase
+ x_val, y_val is used for validation phase
+ x_test, y_test is used for testing phase

In [15]:
x_train_gan = split_sequences_gan(train_data, steps, df.columns.get_loc("price_diff"))

x_train, y_train = split_sequences(train_data, steps, df.columns.get_loc("price_diff"))
x_val, y_val = split_sequences(val_data, steps, df.columns.get_loc("price_diff"))
x_test, y_test = split_sequences(test_data, steps, df.columns.get_loc("price_diff"))

Define the number of outputs for LSTM

In [16]:
num_of_outputs = 64

## **Original model**

Define loss functions

In [17]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()
mae = tf.keras.losses.MeanAbsoluteError()

Define losses for discriminator and generator

In [18]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return 3*mae(y_train.reshape((-1,1)),generator(x_train)) + cross_entropy(tf.ones_like(fake_output), fake_output)

Define adam optimizer

In [19]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.0001
)

### **1. Generator**

In [20]:
class GeneratorModel(Model):
    def __init__(self, num_of_outputs:int):
        super().__init__()
        self.t2v_first_col = Dense(1, input_shape=(steps,df.shape[-1]), activation=None)
        self.t2v_others_col = Dense(255, input_shape=(steps,df.shape[-1]), activation=None)
        self.LSTM1 = LSTM(num_of_outputs, return_sequences=True, recurrent_dropout=0.3)
        self.LSTM2 = LSTM(num_of_outputs, return_sequences=True, recurrent_dropout=0.3)
        self.LSTM3 = LSTM(num_of_outputs, return_sequences=False, recurrent_dropout=0.3)
        self.batch_norm1 = BatchNormalization()
        self.batch_norm2 = BatchNormalization()
        self.Dense1 = Dense(32)
        self.Dense2 = Dense(1)
        
    def call(self, inputs):
        t2v_x1 = self.t2v_first_col(inputs)
        t2v_x2 = tf.sin(self.t2v_others_col(inputs))
        t2v = tf.concat([t2v_x1,t2v_x2],-1)
        x1 = self.LSTM1(t2v)
        x1 = self.batch_norm1(x1)
        x2 = self.LSTM2(x1)
        x2 = self.batch_norm1(x2)
        x3 = self.LSTM3(x2)
        x4 = self.Dense1(x3)
        x5 = self.batch_norm2(x4)
        return self.Dense2(x5)
        
        
generator = GeneratorModel(num_of_outputs=num_of_outputs)

2022-06-16 22:09:47.071324: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-16 22:09:47.073407: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:988] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:09:47.073688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:988] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 22:09:47.073831: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:988] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so ret

### **2. Discriminator**

In [21]:
class DiscriminatorModel(Model):
    def __init__(self, num_of_outputs:int):
        super().__init__()
        self.Reshape = Reshape((steps+1,1))
#         self.Bidirectional_LSTM1 = Bidirectional(LSTM(num_of_outputs, return_sequences=False))
        self.batch_norm1 = BatchNormalization()
        self.batch_norm2 = BatchNormalization()
        self.batch_norm3 = BatchNormalization()
        self.batch_norm4 = BatchNormalization()
#         self.Dense1 = Dense(512, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
        self.Dense2 = Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
        self.Dense3 = Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
        self.Dense4 = Dense(1, activation='sigmoid')
        
    def call(self, inputs):
        x0 = self.Reshape(inputs)
#         x1 = self.Bidirectional_LSTM1(x0)
#         x1 = self.batch_norm1(x1)
#         x2 = self.Dense1(x0)
#         x2 = self.batch_norm2(x2)
        x3 = self.Dense2(x0)
        x3 = self.batch_norm3(x3)
        x4 = self.Dense3(x3)
        x4 = self.batch_norm4(x4)
        return self.Dense4(x4)
        
discriminator = DiscriminatorModel(num_of_outputs=num_of_outputs)

In [22]:
def generate_fake_samples(x, generator, price_diff_col_idx):
    pred = generator(x)
    return Concatenate(axis=1)([x[:,:,df.columns.get_loc("price_diff")],pred])

In [23]:
def generate_idx_data(x_train, rate=0.5):
    faked_idx = np.random.randint(0, x_train.shape[0], int(x_train.shape[0]*rate))
    real_idx = np.array(list(set([i for i in range(x_train.shape[0])]).difference(set(faked_idx))))
    return faked_idx, real_idx

In [24]:
generator_optimizer = adam
discriminator_optimizer = adam

In [25]:
generator.compile(loss="mae")
for i in range(5):
    print("i =", i, ":")
    faked_idx, real_idx = generate_idx_data(x_train)
    x_real = x_train_gan[real_idx]
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        x_faked = generate_fake_samples(x_train[faked_idx], generator, df.columns.get_loc("price_diff"))
        real_output = discriminator(x_real, training=True)
        fake_output = discriminator(x_faked, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    print("Generator loss %s | Discriminator loss %s" %(float(gen_loss), float(disc_loss)))
    print("Val loss evaluation", generator.evaluate(x_val,y_val))
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

i = 0 :


2022-06-16 22:09:49.841085: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-06-16 22:10:00.074483: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.83MiB (rounded to 2970880)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-06-16 22:10:00.074671: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2022-06-16 22:10:00.074698: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 34, Chunks in use: 34. 8.5KiB allocated for chunks. 8.5KiB in use in bin. 4.1KiB client-requested in use in bin.
2022-06-16 22:10:00.074709: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512

ResourceExhaustedError: Exception encountered when calling layer "lstm_cell_2" (type LSTMCell).

{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2]

Call arguments received by layer "lstm_cell_2" (type LSTMCell):
  • inputs=tf.Tensor(shape=(11605, 64), dtype=float32)
  • states=('tf.Tensor(shape=(11605, 64), dtype=float32)', 'tf.Tensor(shape=(11605, 64), dtype=float32)')
  • training=False

In [ ]:
predictions_train = generator.predict(x_train)
predictions_train = data_denormalization(predictions_train, original_df=train_df, steps=steps, col_idx=df.columns.get_loc("price_diff"))
y_real_train = train_df["price_diff"][20+steps:].reset_index(drop=True)
print("Train Accuracy:",((y_real_train - y_real_train.shift(1))*(predictions_train.to_numpy()-y_real_train.shift(1).to_numpy())>0).sum()/predictions_train.shape[0])

In [ ]:
predictions_val = generator.predict(x_val)
predictions_val = data_denormalization(predictions_val, original_df=val_df, steps=steps, col_idx=df.columns.get_loc("price_diff"))
y_real_val = val_df["price_diff"][20+steps:].reset_index(drop=True)
print("Validation Accuracy:",((y_real_val - y_real_val.shift(1))*(predictions_val.to_numpy()-y_real_val.shift(1).to_numpy())>0).sum()/predictions_val.shape[0])

In [ ]:
predictions_test = generator.predict(x_test)
predictions_test = data_denormalization(predictions_test, original_df=test_df, steps=steps, col_idx=df.columns.get_loc("price_diff"))
y_real_test = test_df["price_diff"][20+steps:].reset_index(drop=True)
print("Test Accuracy:",((y_real_test - y_real_test.shift(1))*(predictions_test.to_numpy()-y_real_test.shift(1).to_numpy())>0).sum()/predictions_test.shape[0])

In [ ]:
plt.figure(figsize=(16,9), dpi=90)
plt.plot(np.arange(100), predictions_test[0:100], label="predictions")
plt.plot(np.arange(100), y_real_test[0:100], label="real")
plt.legend()  
plt.show()